## DATA ACQUISITION

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import requests

# Import 2020 data
url_2020 = "https://raw.githubusercontent.com/glyph/election-stats/refs/heads/main/vote_share_2020.csv"
results_2020 = pd.read_csv(url_2020)
state_list = results_2020.state
swing_states = ["MI", "WI", "PA", "GA", "AZ", "NV", "NC"]

# Import 2024 data
harris_count = []
trump_count = []
pct_counted = []

for abbrev in state_list:
    url = "https://politics.api.cnn.io/results/race/2024-PG-" + abbrev + ".json"
    json = requests.get(url).json()
    cand1_count = json.get("candidates")[0].get("voteNum")
    cand2_count = json.get("candidates")[1].get("voteNum")
    pct_counted.append(json.get("percentReporting"))
    if json.get("candidates")[0].get("candidatePartyCode") == "D":
        harris_count.append(cand1_count)
        trump_count.append(cand2_count)
    else:
        trump_count.append(cand1_count)
        harris_count.append(cand2_count)

results_2024 = pd.DataFrame(
    {
        "state": state_list,
        "harris": harris_count,
        "trump": trump_count,
        "pct_counted": pct_counted,
    }
)

## DATA WRANGLING

In [2]:
# Project the final 2024 results from existing tallies
results_2024["harris_proj"] = round(
    results_2024.harris / results_2024.pct_counted * 100
)
results_2024["trump_proj"] = round(results_2024.trump / results_2024.pct_counted * 100)


# Add state status column to 2024 data
def check_status(us_state):
    if us_state in swing_states:
        return "swing"
    else:
        return "safe"


results_2024["status"] = results_2024["state"].apply(check_status)

# Combine data for 2020 and 2024
results_both = pd.merge(results_2024, results_2020, on="state")

# Get totals by candidate and state status
safe_results = results_both[results_both["status"] == "safe"]
swing_results = results_both[results_both["status"] == "swing"]
biden_total_safe = np.sum(safe_results["d_2020_total"])
biden_total_swing = np.sum(swing_results["d_2020_total"])
trump_20_total_safe = np.sum(safe_results["r_2020_total"])
trump_20_total_swing = np.sum(swing_results["r_2020_total"])
harris_total_safe = np.sum(safe_results["harris"])
harris_total_swing = np.sum(swing_results["harris"])
trump_24_total_safe = np.sum(safe_results["trump"])
trump_24_total_swing = np.sum(swing_results["trump"])
harris_proj_total_safe = np.sum(safe_results["harris_proj"])
harris_proj_total_swing = np.sum(swing_results["harris_proj"])
trump_24_proj_total_safe = np.sum(safe_results["trump_proj"])
trump_24_proj_total_swing = np.sum(swing_results["trump_proj"])

## VOTE CHANGE BY STATUS

### CURRENT

In [3]:
# Calculate percent change by candidate and state status
possible_status = ["safe", "swing"]
harris_shifts = [
    round((harris_total_safe / biden_total_safe * 100), 2) - 100,
    round((harris_total_swing / biden_total_swing * 100), 2) - 100,
]
trump_shifts = [
    round((trump_24_total_safe / trump_20_total_safe * 100), 2) - 100,
    round((trump_24_total_swing / trump_20_total_swing * 100), 2) - 100,
]

shift_pcts = pd.DataFrame(
    {"status": possible_status, "harris": harris_shifts, "trump": trump_shifts}
)

print(shift_pcts)

  status  harris  trump
0   safe   -9.88   3.36
1  swing   -0.31   6.26


### PROJECTED

In [4]:
# Calculate percent change by candidate and state status
possible_status_2 = ["safe", "swing"]
harris_shifts_2 = [
    round((harris_proj_total_safe / biden_total_safe * 100), 2) - 100,
    round((harris_proj_total_swing / biden_total_swing * 100), 2) - 100,
]
trump_shifts_2 = [
    round((trump_24_proj_total_safe / trump_20_total_safe * 100), 2) - 100,
    round((trump_24_proj_total_swing / trump_20_total_swing * 100), 2) - 100,
]

shift_pcts_proj = pd.DataFrame(
    {"status": possible_status_2, "harris": harris_shifts_2, "trump": trump_shifts_2}
)

print(shift_pcts_proj)

  status  harris  trump
0   safe   -8.65   4.70
1  swing    0.70   7.33


## VOTE CHANGE BY STATE

### CURRENT

In [5]:
# Calculate percent change by candidate in each swing state
possible_status_3 = ["safe"]
harris_shifts_3 = [round((harris_total_safe / biden_total_safe * 100), 2) - 100]
trump_shifts_3 = [round((trump_24_total_safe / trump_20_total_safe * 100), 2) - 100]
for abbrev in swing_states:
    state_pop_vote = results_both[results_both["state"] == abbrev]
    possible_status_3.append(abbrev)
    harris_shifts_3.append(
        round(
            np.sum(state_pop_vote["harris"])
            / np.sum(state_pop_vote["d_2020_total"])
            * 100,
            2,
        )
        - 100
    )
    trump_shifts_3.append(
        round(
            np.sum(state_pop_vote["trump"])
            / np.sum(state_pop_vote["r_2020_total"])
            * 100,
            2,
        )
        - 100
    )

shift_pcts_state = pd.DataFrame(
    {"status": possible_status_3, "harris": harris_shifts_3, "trump": trump_shifts_3}
)

print(shift_pcts_state)

  status  harris  trump
0   safe   -9.88   3.36
1     MI   -2.41   6.29
2     WI    2.29   5.43
3     PA   -1.01   4.91
4     GA    3.01   8.18
5     AZ   -5.34   6.53
6     NV    0.24  12.14
7     NC    1.16   5.06


### PROJECTED

In [6]:
# Calculate percent change by candidate in each swing state
possible_status_4 = ["safe"]
harris_shifts_4 = [round((harris_proj_total_safe / biden_total_safe * 100), 2) - 100]
trump_shifts_4 = [
    round((trump_24_proj_total_safe / trump_20_total_safe * 100), 2) - 100
]
for abbrev in swing_states:
    state_pop_vote = results_both[results_both["state"] == abbrev]
    possible_status_4.append(abbrev)
    harris_shifts_4.append(
        round(
            np.sum(state_pop_vote["harris_proj"])
            / np.sum(state_pop_vote["d_2020_total"])
            * 100,
            2,
        )
        - 100
    )
    trump_shifts_4.append(
        round(
            np.sum(state_pop_vote["trump_proj"])
            / np.sum(state_pop_vote["r_2020_total"])
            * 100,
            2,
        )
        - 100
    )

shift_pcts_state_proj = pd.DataFrame(
    {"status": possible_status_4, "harris": harris_shifts_4, "trump": trump_shifts_4}
)

print(shift_pcts_state_proj)

  status  harris  trump
0   safe   -8.65   4.70
1     MI   -1.42   7.37
2     WI    3.32   6.50
3     PA   -0.01   5.97
4     GA    4.05   9.27
5     AZ   -4.38   7.61
6     NV    1.26  13.27
7     NC    2.18   6.12


## VOTE SHARE

In [7]:
# Determine Democratic candidate share of 2-party vote in all configurations
d_2020_share_swing = round(
    biden_total_swing / (biden_total_swing + trump_20_total_swing) * 100, 2
)
d_2020_share_safe = round(
    biden_total_safe / (biden_total_safe + trump_20_total_safe) * 100, 2
)
d_2024_share_swing_current = round(
    harris_total_swing / (harris_total_swing + trump_24_total_swing) * 100, 2
)
d_2024_share_safe_current = round(
    harris_total_safe / (harris_total_safe + trump_24_total_safe) * 100, 2
)
d_2024_share_swing_proj = round(
    harris_proj_total_swing
    / (harris_proj_total_swing + trump_24_proj_total_swing)
    * 100,
    2,
)
d_2024_share_safe_proj = round(
    harris_proj_total_safe / (harris_proj_total_safe + trump_24_proj_total_safe) * 100,
    2,
)

# Organize data and create a data frame
elections = ["2020 Final", "2024 Current", "2024 Projected"]
safe_list = [d_2020_share_safe, d_2024_share_safe_current, d_2024_share_safe_proj]
swing_list = [d_2020_share_swing, d_2024_share_swing_current, d_2024_share_swing_proj]
vote_share_df = pd.DataFrame(
    {"Election": elections, "Safe States": safe_list, "Swing States": swing_list}
)

print(vote_share_df)

         Election  Safe States  Swing States
0      2020 Final        52.73         50.39
1    2024 Current        49.31         48.79
2  2024 Projected        49.32         48.79
